# STA130 Tutorial 5: Hypothesis Testing

# Today's agenda (5 min)

- Hypothesis test walk-through (can emphasize/explain vocab here)

- Group Presentation

- Q&A/Vocabulary list (if time allows)

- Type I/II error discussion (if time allows, on a separate handout)

## Worked Example: Wheel of Destiny (Introduce Problem)

Stella McStat had been running a small-time gambling operation on campus for several months during her first year at U of T. It was disrupted during COVID, but now that vaccines are mandatory to be on campus, Stella is getting things back up and running.

<p align="center">
  <img src="stella2.png" />
  <br/>Adapted from Lawton, L. (2009) An Exercise for Illustrating the Logic of Hypothesis Testing, Journal of Stat. Education, 17(2).
</p>


The format: For each spin of the wheel, two gamblers take part. For a toonie each ($2 Canadian), Stella sells one a red ticket and one a black ticket  (i.e., total $4). Then Stella spins the Wheel of Destiny. The person who holds the colour on which the spinner stops gets $3.50 (Stella keeps $0.50 per spin for running the game and providing snacks).

Stella just bought a new spinner, the critical piece of equipment for this game. She's heard some mixed reviews about the manufacturer she has purchased from. Before she beings using this spinner, she wants to make sure that it is, in fact, fair—she wants both colours to come up equally often. Because of the set-up of the game, Stella has no incentive to cheat and wants the game to be as fair as possible.

Everything she can examine about the wheel seems fine; there is the same number of sectors of each colour and they each have the same area. BUT! Stella has a great idea and decides to come to YOU, her statistical guru, and ask you to verify that the new spinner is fit to use.

<p align="center">
  <img src="fair.png" />
  <br/>Adapted from Lawton, L. (2009) An Exercise for Illustrating the Logic of Hypothesis Testing, Journal of Stat. Education, 17(2).
</p>


## Discussion 1 (20 minutes)
- How could we use hypothesis testing to solve this problem?

- What is a population?

- What is the null hypothesis in this case? The alternate hypothesis?

- What is a (one-sample) permutation test? How might we go about conducting a one-sample permutation test?

    - What's the difference between a **parametric** and **nonparametric** test?

    - What's the difference between a **dependent** and **independent** sample?
    
- What is a p-value?


    

## Discussion 1 Ideas (20 minutes)
**Statistical Inference for a Single Sample Proportion**: 

 1. State null hypothesis $H_0$ and alternative hypothesis $H_A$ 

 2. Calculate a *test statistic* based on one observed *random sample* 

 3. *Simulate* samples under $H_0$ and calculate the statictic for the simulated sample. Repeat many times. (this constitutes our estimated *sampling distribution*)

 4. Evaluate the evidence against $H_0$ by calculating the *p-value*

 5. Make a conclusion (based on the strength of evidence against $H_0$)

$
\begin{align}
H_0 &{}: {} p = 0.5 \quad\text{probability of spinning one color(like red)}\\
H_1 &:{} p \neq 0.5 \quad\text{(alternative hypothesis)}\\\\
\end{align}$  
We do not know the population for the probability distribution, so we can't assume anything about the distribution of the population, or the *data distribution*.


To perform our one-sample permutation test, we could perform some **actual** spins and calculate the proportion of landing red 
to find our observed test statistic.  

In [1]:
# Getting observed test statistic
import numpy as np
np.random.seed(1)
spins = 1000  # chose 1000 spins, pros/cons of doing more/less?
spin_results = np.random.choice(['Red', 'Black'], size=spins, replace=True, p = [0.5, 0.5])  # Lets assume the spinner is truthfully fair (has 50-50 to spin red). What if this wasn't the case?
numeric_spin_results = np.where(spin_results == 'Red', 1, 0)  # convert spin results to 1's for red and 0's for black
test_stat = numeric_spin_results.sum() / spins
print('Our test statistic is ' + str(test_stat) + ', meaning we spun ' + str(int(test_stat * spins)) + ' reds out of ' + str(spins) + ' spins.')

Our test statistic is 0.494, meaning we spun 494 reds out of 1000 spins.


We can then simulate a sampling distribution under the assumption of our null hypothesis. This is a nonparametric permutation test as there are no normality assumptions about sampling distribution. It is also an example of independent sampling, as each bootstrap sample does not depend on the values of other bootstrap samples.

In [8]:
import pandas as pd
import plotly.express as px
np.random.seed(seed=1)
num_samples = 1000  # how many bootstrap samples should we take?
sample_size = 1000  # size of each bootstrap sample?
sample_proportions = []
for i in range(num_samples):
    sample = np.random.choice(['Red', 'Black'], size=sample_size, replace=True)
    sample_proportions += [np.count_nonzero(sample == 'Red') / sample_size]
fig = px.histogram(pd.DataFrame({'Proportion of Red': sample_proportions}), x='Proportion of Red', 
                   title='Proportion of Red Sampling Distribution')
fig.show()


The above graph is a *sampling distribution*, which is the distribution of our test statistic(proportion of red) through repeated sampling of a population. This differs from looking at the distribution for a given population, or the *data distribution* (usually looks like a curve opposed to a histogram) 

Calculating a p-value (the probability of a test statistic being as or more extreme than the observed test statistic if the null hypothesis was true) will help us determine whether we have evidence against the null hypothesis.  

In [9]:
num_more_extreme = (abs(np.array(sample_proportions) - 0.5) >= abs(test_stat - 0.5)).sum()
p_value = num_more_extreme / num_samples
print(str(num_more_extreme) + ' sample statistics were as or more extreme than our observed test statistic of ' + str(test_stat) + ', giving us a p-value of ' + str(p_value))

705 sample statistics were as or more extreme than our observed test statistic of 0.494, giving us a p-value of 0.705


<p align="center">
  <img src="https://www.jcpcarchives.org/userfiles/values-of-p-Inference.jpg" />
</p>

Since our p-value is more than 0.10, we have no evidence against the null hypothesis of the probability of spinning red is 0.5

## Discussion 2: Different Types of P-Values (30 minutes)
### One sample t-test
- Parametric null hypothesis assumption

- Estimate standard deviation ${\sigma}$ of null hypothesis with the sample standard deviation ${\hat\sigma}$

    - Hat symbolizes estimation
<br/><br/>
- Allows us to theoretically derive sampling distribution of test statistic, so p-value derived *theoretically*

    - P-value has same definition, but quantified by integrating area under theoretical distribution curve, rather than counting values as in our simulation based approach
    
    - Relies on parametric assumption

    - How theoretical derivation works for later stats courses

Exp. Our null hypothesis for the "Wheel of Destiny" problem becomes  
$
\begin{align}
H_0 &: {} \mu = 8 \quad\text{(null hypothesis)}\\
&\color{white}{:} \text{ } \sigma \text{ estimated as sample standard deviation } \hat{\sigma} \\
&\color{white}{:} {} \text{ and the population is normally distributed}\\
H_1 &:{} H_0 \text{ is } \text{False}
\end{align}$
- Need all the assumptions in the null hypothesis to hold for the p-value to make sense
- Different from simulaton approach, where we just needed the assumptions in the null hypothesis to form a sampling distribution, from which a p-value can be calculated from.   

In [10]:
# From "Wheel of Destiny Example"
from scipy import stats
stats.ttest_1samp(numeric_spin_results, 0.5)

TtestResult(statistic=-0.37931084646688906, pvalue=0.7045376554833558, df=999)

We observe a theoritical t-test based p-value of 0.705... which again provides no evidence against the null hypothesis of the probability of spinning red is 0.5

### Median Sign Test

- Null hypothesis about the median

- By assumption of median, half of the sample will be greater than and half will be less than the null hypothesis assumed median

    - Implies binomial sampling distribution (a probability distribution that takes on one of two independent values)
    
    - Nonparametric test, no parametric assumption as we know the sampling distribution! 
    
- "Sign test" puts negative sign on samples less than median and positive sign on samples more than median, should be a 50-50 split. 
We can derive a sampling distribution from this test, calculate
    


Exp. Our null hypothesis for the "Wheel of Destiny" problem becomes  
$
\begin{align}
H_0 &: {} Median = 0.5\\
H_1 &:{} H_0 \text{ is } \text{False}
\end{align}$


In [11]:
spins_greater_than = ((numeric_spin_results > 0.5).sum()+(numeric_spin_results >= 0.5).sum())/2  # out of 1000
percent = (spins_greater_than / spins) * 100
print(str(int(spins_greater_than)) + ' samples out of ' + str(spins) + ' are greater than the median, for ' + str(percent) + '%')

494 samples out of 1000 are greater than the median, for 49.4%


In [12]:
# sign test for theoretical p-value. Relies on knowledge of binomial, cumulative distribution and sign test
(stats.binom(n=spins,p=0.5).cdf(spins_greater_than-1))*2

0.6810229832764916

Our theoretical p-value is 0.681... which again provides no evidence against the null hypothesis of the probability of spinning red is 0.5

### Wilcoxon Signed Ranked Test
- Nonparametric sign test (no distribution assumptions)

- Compare medians of two dependent samples to see if they are statistically significantly different from each other

    - Dependent samples signifies subjects in one group provide information about subjects in the other group
<br/><br/>
- The "signed ranked test" ranks the differences between the two observations (if your two samples have size n, rank 1 given to smallest difference while rank n given to largest difference). Then applies negative sign when the difference is negative (and positive sign when difference positive)
    
    - Tests whether the mean of these signed ranks is 0
<br/><br/>

Exp. Is there a difference in the 6th grade and 7th grade exam scores for a group of students?  
$
\begin{align}
H_0 &: {} \text{median}_{grade 6} - \text{median}_{grade 7} = 0\\
H_1 &:{} H_0 \text{ is } \text{False}
\end{align}$

In [85]:
import random
random.seed(1)
# Assume score populations are given by below code
grade_6_exam_scores = [random.randint(30, 70) for i in range(0, 100)]
grade_7_exam_scores = [random.randint(50, 80) for i in range(0, 100)]
diff = [grade_7_exam_scores[i]-grade_6_exam_scores[i] for i in range(len(grade_6_exam_scores))]  # difference in scores between grade 7 and grade 6
diff  # most students appear to have scored better in grade 7

[23,
 14,
 43,
 22,
 24,
 3,
 21,
 -2,
 17,
 24,
 33,
 19,
 42,
 -4,
 5,
 7,
 47,
 18,
 33,
 34,
 13,
 37,
 16,
 44,
 23,
 35,
 10,
 37,
 2,
 20,
 23,
 20,
 2,
 33,
 3,
 7,
 2,
 12,
 28,
 22,
 19,
 6,
 28,
 30,
 7,
 -4,
 14,
 26,
 -3,
 21,
 38,
 18,
 -2,
 7,
 7,
 8,
 26,
 9,
 3,
 -6,
 5,
 13,
 -12,
 45,
 -8,
 30,
 12,
 19,
 36,
 23,
 14,
 5,
 16,
 18,
 18,
 35,
 12,
 -11,
 22,
 -3,
 3,
 19,
 14,
 42,
 9,
 -12,
 -9,
 -14,
 20,
 29,
 15,
 -1,
 15,
 22,
 13,
 -9,
 -7,
 22,
 25,
 -7]

In [81]:
stats.wilcoxon(diff)

WilcoxonResult(statistic=361.0, pvalue=9.928135244444676e-14)

We observe a p-value < 0.001... which provides very strong evidence against the null hypothesis that there is no difference between grade 6 and grade 7 test scores of the same students

## Discussion 3 (10 minutes)
#### Similarities between all 3 tests? Differences?

<!-- - **Statistical Inference for a Single Sample Proportion**: -->

<!--   1. State null hypothesis $H_0$ and alternative hypothesis $H_1$ -->

<!--   2. Calculate a *test statistic* based on one observed *random sample* -->

<!--   3. *Simulate* samples under $H_0$ and calculate the statictic for the simulated sample. Repeat many times. (this constitutes our estimated *sampling distribution*) -->

<!--   4. Evaluate the evidence against $H_0$ by calculating the *p-value* -->

<!--   5. Make a conclusion (based on the strength of evidence against $H_0$) -->

# Group Presentation (30 min) :

*I will post the following material on Quercus in advance for students to access*

You are an amateur data detective that reads studies in your free time and checks whether the researchers conform to academic integrity. Here are some studies that you recently found. Identify the hypotheses (One-Group or Two-Group) of the researchers and describe how you could validate the findings using a simulation test. Make a presentation on your hypothesis test.

### SCENARIOS

(a) A health survey asked 200 individuals aged 20-45 living in Toronto to report the number minutes they exercised last week. Researchers were interested in determining whether the average duration of exercise differed between people who consume cannabis and those who do not consume cannabis. Assume the researchers who conducted this study found that people who used cannabis exercised, on average, 20 minutes per week. In contrast, people who did not use cannabis exercised 40 minutes per week, on average. The researchers reported a p-value of 0.249.

(b) A study was conducted to examine whether a mother's exposure to alcohol while she was pregnant affects the odds of developemntal disabilities. The researchers used a birth registry of all children born in Ontario in 2018, which included approximately 130,000 births. The researchers found that 0.6% of mothers reported that their babies have developmental disabilities and 56% of babies born to mothers who consumed alcohol at least once during pregnancy. In contrast, 55% of healthy babies were born to mothers who were not exposed to alcohol. The researchers reported a p-value of 0.50.

(c) A group of researchers wanted to find out whether completing a degree in Statistics is a worthy inverstment for an individual. They conducted a survey of graduates from the University of Toronto and included 1,000 recent graduates who completed their Bachelor’s degree in the last five years in the study; 80% of the respondents were female and 20% were male.The median reported income of a UofT graduate was \\$76,000. Among statistics graduates, the median reported income was \\$84,000. The researchers reported a p-value of 0.014.

(d) A team of researchers were interested in understanding millennial’s views regarding housing affordability in Toronto. The team interviewed 850 millennials currently living in Toronto. 84% reported that they felt housing prices were unaffordable in the city. Suppose the researchers were interested in testing whether this proportion was different from a study published last year, which found that 92% of millennials reported that housing costs were unaffordable. The researchers reported a p-value of 0.023.

(e) Suppose a drug company was interested in testing a new weight-loss drug. They enrolled 20,000 participants and assigned 10,000 to take their new drug, SlimX, and 10,000 to take a placebo. The researchers found that over 2 months, participants who took SlimX lost, on average, 5 lbs. In comparison, the control group lost 4.5 lbs during the same time. The researchers reported a p-value of <0.0001.

### In your presentation, it is important to:  

- Give an introduction of the question
- State the hypotheses $H_0$ and $H_A$ 
- Specify the assumption(s) of your hypothesis test, if any
- Describe the procedures for data collection (sampling) and simulation
- Include and explain at least two terms from the vocabulary
- Summarize your findings and interpret the results 
- Provide limitations (optional but good practice) (e.g. sample size, study design issue)

## Some things to keep in mind  

- The introduction should state the overall purpose of the presentation   

- Each main point should be “signaled.” Use *signal phrases* or *signposts* to transition between your points.

- Every group member must speak during the presentation or marks will be deducted for participation.


## Vocabulary (If Time)
- statistical inference 
- population 
- random sample 
- sampling distribution 
- simulation 
- parameter 
- simulation statistic 
- test statistic 
- p-value

#### This Week's Vocab (15-20 min) :

- Vocabs required for the group presentation 
- Type 1 and 2 errors (also written as Type I and II errors)
- Comparing two population means/medians/proportions
- One-group hypothesis tests
- sample
- for() loops
- diff()

## Type I/II Error Discussion (If Time)
[W4_Optional-discussion-group-prompt.pdf](./W4_Optional-discussion-group-prompt.pdf)